## Preperation: OpenAI API KEY

In [7]:
import json

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("OPENAI_API_KEY"))  # should show your key


## Agent1: Domain Detector

In [3]:
!pip install -U langchain-core langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
!pip install -U langgraph

In [ ]:
!pip install langchain-openai langchain-core langgraph

### TBF-Test 0 for Agent_D: with a fixed knowledge database

### TBF-Test 1 for Agent_D: use LLM to dectect domain

In [8]:
from agents.domain_detector_v0 import run_domain_detector
import json

columns = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
column_types = {
    "Quarter": "object",
    "number_Customers": "int",
    "Total_Transactions": "float",
    "Revenue": "float",
    "Profit": "float"
}

domains = run_domain_detector(columns, column_types)
print(json.dumps(domains, indent=2))




[
  {
    "domain": "Business Analytics",
    "jargon_terms": [
      "KPI",
      "ROI",
      "Forecasting",
      "Segmentation",
      "Data Mining"
    ]
  },
  {
    "domain": "Business Finance",
    "jargon_terms": [
      "ROI",
      "EBITDA",
      "Liquidity",
      "Cash Flow",
      "Capital Expenditure"
    ]
  },
  {
    "domain": "Business Economics",
    "jargon_terms": [
      "demand_curve",
      "elasticity",
      "marginal_cost",
      "opportunity_cost",
      "market_share"
    ]
  },
  {
    "domain": "Business Accounting",
    "jargon_terms": [
      "ledger",
      "depreciation",
      "accrual",
      "amortization",
      "equity"
    ]
  },
  {
    "domain": "Business Management",
    "jargon_terms": [
      "KPI",
      "ROI",
      "SWOT",
      "P&L",
      "Forecast"
    ]
  }
]


### 🧹 Step 1. Cleaned Domain–Concept Pairs
- Remove redundant jargon terms across domains.
- Only retain the first occurrence of each term to ensure one-to-one mapping.

In [14]:
from collections import Counter

# keep the first occurrence of any jargon term;
# drop later duplicates that show up in other domains.

def get_first_unique_pairs(domains_found):
    """
    Keep the first‑seen (domain, jargon) pair for every jargon_term.
    Later occurrences of the same jargon_term in other domains are skipped.
    """
    seen = set()
    unique_pairs = []

    for entry in domains_found:          # keep original order
        dom = entry["domain"]
        for term in entry["jargon_terms"]:
            if term not in seen:         # only accept the first time we see it
                unique_pairs.append({"domain": dom, "jargon_terms": term})
                seen.add(term)

    return unique_pairs

cleaned = get_first_unique_pairs(domains)
import json, pprint; pprint.pp(cleaned, width=60)

[{'domain': 'Business Analytics', 'jargon_terms': 'KPI'},
 {'domain': 'Business Analytics', 'jargon_terms': 'ROI'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Forecasting'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Segmentation'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Data Mining'},
 {'domain': 'Business Finance', 'jargon_terms': 'EBITDA'},
 {'domain': 'Business Finance',
  'jargon_terms': 'Liquidity'},
 {'domain': 'Business Finance',
  'jargon_terms': 'Cash Flow'},
 {'domain': 'Business Finance',
  'jargon_terms': 'Capital Expenditure'},
 {'domain': 'Business Economics',
  'jargon_terms': 'demand_curve'},
 {'domain': 'Business Economics',
  'jargon_terms': 'elasticity'},
 {'domain': 'Business Economics',
  'jargon_terms': 'marginal_cost'},
 {'domain': 'Business Economics',
  'jargon_terms': 'opportunity_cost'},
 {'domain': 'Business Economics',
  'jargon_terms': 'market_share'},
 {'domain': 'Business Accounting',
  'jargon_terms': 'ledger'},
 {'dom

### 🧪 Step 2. Filter by Relevance & Formula
Use an LLM to evaluate whether each term is:
- Relevant to the current dataset columns, and， Expressible as a formula (e.g., ROI = Profit / Cost).
- Only keep terms that meet both criteria.

In [15]:
# ------------ 1. imports & chat object ------------
from langchain_openai import ChatOpenAI
from typing import List, Dict, Union

# feel free to tweak these
MODEL_NAME   = "gpt-4o"   # or "gpt-4o", "gpt-3.5-turbo"
TEMPERATURE  = 0
MAX_TOKENS   = 6               # we only need YES / NO
API_KEY_ENV  = "OPENAI_API_KEY"

# singleton chat object (re‑used for every call)
chat = ChatOpenAI(
    model_name   = MODEL_NAME,
    temperature  = TEMPERATURE,
    max_tokens   = MAX_TOKENS,
    openai_api_key = None,     # uses env var by default
)


# ------------ 2. prompt builder ------------
def _build_prompt(jargon:str, domain:str,
                  columns:List[str], column_types:Dict[str,str]) -> str:
    few_shots = """
### Example 1
Data columns: ["Date","Open","High","Low","Close"]
Column types: {"Date":"object","Open":"float","High":"float","Low":"float","Close":"float"}
Jargon term: "EBITDA"
Answer: YES, YES

### Example 2
Data columns: ["User_ID","Session_Duration","Clicks"]
Column types: {"User_ID":"int","Session_Duration":"float","Clicks":"int"}
Jargon term: "Exploratory Data Analysis"
Answer: YES, NO

### Example 3
Data columns: ["Temperature","Humidity"]
Column types: {"Temperature":"float","Humidity":"float"}
Jargon term: "Sharpe Ratio"
Answer: NO, YES
""".strip()

    rule = (
        "First flag = YES if the term is related to the given columns; else NO.\n"
        "Second flag = YES only if the term is normally written as a numeric or algebraic "
        "equation (e.g. ROI = Profit/Cost); else NO.\n"
        "Answer format: YES/NO,YES/NO ‑ no extra words."
    )

    task = f"""
### Task
Domain      : {domain}
Data columns: {json.dumps(columns)}
Column types: {json.dumps(column_types)}
Jargon term : "{jargon}"
{rule}
Answer:"""

    return f"{few_shots}\n{task}"

# ------------- 3. evaluator -------------
def lc_relevance(
        pairs: List[Dict[str, str]],
        columns: List[str],
        column_types: Dict[str, str],
        *,
        keep_only_formula: bool = True,   # <-- new switch
) -> List[Dict[str, Union[str, bool]]]:
    """
    Evaluate every jargon term and (optionally) RETURN ONLY THOSE
    that are both   relevant == True   AND   has_formula == True.

    Set `keep_only_formula=False` if you still want the full list.
    """
    good_rows: list[dict] = []
    all_rows : list[dict] = []

    for p in pairs:
        term   = p.get("jargon_terms") or p.get("concept")
        domain = p["domain"]

        prompt = _build_prompt(term, domain, columns, column_types)
        reply  = chat.invoke(prompt).content.strip().upper()

        # Accept variants such as "YES , NO"
        parts       = [x.strip() for x in reply.split(",")]
        rel_flag    = parts[0].startswith("Y") if parts else False
        formula_flag = parts[1].startswith("Y") if len(parts) > 1 else False

        row = {
            "domain"      : domain,
            "jargon_term" : term,
            "relevant"    : rel_flag,
            "has_formula" : formula_flag,
        }

        all_rows.append(row)
        if rel_flag and formula_flag:
            good_rows.append(row)

    return good_rows if keep_only_formula else all_rows


In [16]:
import pandas as pd

# --- Evaluate every jargon term ------------------------------

evaluated_all = lc_relevance(
        cleaned,
        columns,
        column_types,
        keep_only_formula=False   # return *all* rows so we can inspect the two flags
)

# --- Put results in a nice, readable table -------------------
import pandas as pd
from IPython.display import display  # Jupyter‑friendly

# convert list[dict] → DataFrame
results_df = pd.DataFrame(evaluated_all)

# reorder columns for clarity (optional)
col_order = [
    "domain",
    "jargon_term",
    "relevant",
    "has_formula",
]
results_df = results_df[col_order]

# show the DataFrame
print("\n===== evaluator output (full list) =====\n")
print(results_df.to_string(index=False))  # nice console print




===== evaluator output (full list) =====

             domain         jargon_term  relevant  has_formula
 Business Analytics                 KPI      True        False
 Business Analytics                 ROI      True         True
 Business Analytics         Forecasting      True        False
 Business Analytics        Segmentation      True        False
 Business Analytics         Data Mining      True        False
   Business Finance              EBITDA      True         True
   Business Finance           Liquidity      True         True
   Business Finance           Cash Flow      True         True
   Business Finance Capital Expenditure      True        False
 Business Economics        demand_curve      True        False
 Business Economics          elasticity      True         True
 Business Economics       marginal_cost      True         True
 Business Economics    opportunity_cost      True        False
 Business Economics        market_share      True         True
Business Acc

### 🧰 Step 3. Prepare for Formula Retriever
- Return a cleaned list of domain–jargon pairs that passed filtering.
- This output serves as input for the Formula Retriever module.

In [17]:
evaluated = lc_relevance(cleaned, columns, column_types)

print(json.dumps(evaluated, indent=2))

[
  {
    "domain": "Business Analytics",
    "jargon_term": "ROI",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "EBITDA",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "Liquidity",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "Cash Flow",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Economics",
    "jargon_term": "elasticity",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Economics",
    "jargon_term": "marginal_cost",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Economics",
    "jargon_term": "market_share",
    "relevant": true,
    "has_formula": true
  }
]


In [18]:
print(evaluated)

[{'domain': 'Business Analytics', 'jargon_term': 'ROI', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'EBITDA', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'Liquidity', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'Cash Flow', 'relevant': True, 'has_formula': True}, {'domain': 'Business Economics', 'jargon_term': 'elasticity', 'relevant': True, 'has_formula': True}, {'domain': 'Business Economics', 'jargon_term': 'marginal_cost', 'relevant': True, 'has_formula': True}, {'domain': 'Business Economics', 'jargon_term': 'market_share', 'relevant': True, 'has_formula': True}]


## Agent2: Formula Retriever

### 🧮 Step 1. Invoke Agent 2: Formula Retriever
We call the formula retriever agent to enrich each jargon term with:
- A known numeric formula (if available)
- Definitions of all variables in that formula

In [19]:
from agents.formula_retriever_v0 import get_formula


In [20]:
enriched = get_formula(evaluated)


In [21]:
import json
print(json.dumps(enriched, indent=2))


[
  {
    "domain": "Business Analytics",
    "jargon_term": "ROI",
    "relevant": true,
    "has_formula": true,
    "formula": "(Gain_from_Investment - Cost_of_Investment) / Cost_of_Investment",
    "variables": {
      "Gain_from_Investment": "Total revenue generated from the investment",
      "Cost_of_Investment": "Initial cash outlay for the investment"
    }
  },
  {
    "domain": "Business Finance",
    "jargon_term": "EBITDA",
    "relevant": true,
    "has_formula": true,
    "formula": "Operating_Income + Depreciation + Amortization",
    "variables": {
      "Operating_Income": "Earnings from operations before interest & taxes",
      "Depreciation": "Expense for tangible asset depreciation",
      "Amortization": "Expense for intangible asset amortization"
    }
  },
  {
    "domain": "Business Finance",
    "jargon_term": "Liquidity",
    "relevant": true,
    "has_formula": true,
    "formula": "Current_Assets / Current_Liabilities",
    "variables": {
      "Current_As

In [22]:
# relevant_rows = lc_relevance(pairs, cols, col_types, keep_only_formula=True)

formulas = get_formula(evaluated)

In [23]:
print(formulas)

[{'domain': 'Business Analytics', 'jargon_term': 'ROI', 'relevant': True, 'has_formula': True, 'formula': '(Gain_from_Investment - Cost_of_Investment) / Cost_of_Investment', 'variables': {'Gain_from_Investment': 'The financial gain obtained from the investment', 'Cost_of_Investment': 'The total cost incurred to make the investment'}}, {'domain': 'Business Finance', 'jargon_term': 'EBITDA', 'relevant': True, 'has_formula': True, 'formula': 'Operating_Income + Depreciation + Amortization', 'variables': {'Operating_Income': 'Earnings from operations before interest & taxes', 'Depreciation': 'Expense for tangible asset depreciation', 'Amortization': 'Expense for intangible asset amortization'}}, {'domain': 'Business Finance', 'jargon_term': 'Liquidity', 'relevant': True, 'has_formula': True, 'formula': 'Current_Assets / Current_Liabilities', 'variables': {'Current_Assets': 'Assets that are expected to be converted into cash within one year', 'Current_Liabilities': 'Obligations that are due

In [24]:
df = pd.DataFrame(formulas)
df.head()

,domain,jargon_term,relevant,has_formula,formula,variables
0,Business Analytics,ROI,True,True,(Gain_from_Investment - Cost_of_Investment) / ...,{'Gain_from_Investment': 'The financial gain o...
1,Business Finance,EBITDA,True,True,Operating_Income + Depreciation + Amortization,{'Operating_Income': 'Earnings from operations...
2,Business Finance,Liquidity,True,True,Current_Assets / Current_Liabilities,{'Current_Assets': 'Assets that are expected t...
3,Business Finance,Cash Flow,True,True,Net_Income + Depreciation + Amortization + Cha...,{'Net_Income': 'Total earnings after all expen...
4,Business Economics,elasticity,True,True,(%_Change_in_Quantity_Demanded) / (%_Change_in...,{'%_Change_in_Quantity_Demanded': 'Percentage ...


### 🧹 Step 2. Prepare for Formula Matcher
- Extract deduplicated formulas from enriched output, in order of first appearance.
- This prepares the input for the upcoming Formula Matcher stage.

In [29]:
def extract_formula_keys(items):
    """
    Parameters
    ----------
    items : list[dict]
        The list of dictionaries containing formula information.

    Returns
    -------
    list[str]
        All formulas, deduplicated in order of first appearance.
    """
    seen, out = set(), []
    for d in items:
        formula = d.get("formula")
        if formula and formula not in seen:
            seen.add(formula)
            out.append(formula)
    return out


## 03. Formula Matcher
- This notebook implements Agent 3: Formula Matcher 
- Aligns abstract formula variables (retrieved in the previous step) with actual column names in the dataset. 
- It operates in two stages:

### 1. 🧾 Preparation: Extract Formula List
- We begin by extracting a deduplicated list of all candidate formulas using extract_formula_keys(). 
- These formulas were generated by the Formula Retriever Agent and typically contain abstract variables
- e.g., Gain_from_Investment, Investment_Cost

In [33]:
matcher_variables = extract_formula_keys(formulas)
print(matcher_variables)

['(Gain_from_Investment - Cost_of_Investment) / Cost_of_Investment', 'Operating_Income + Depreciation + Amortization', 'Current_Assets / Current_Liabilities', 'Net_Income + Depreciation + Amortization + Change_in_Working_Capital - Capital_Expenditures', '(%_Change_in_Quantity_Demanded) / (%_Change_in_Price)', 'Change_in_Total_Cost / Change_in_Quantity', 'Company_Sales / Total_Market_Sales']


### 2. 🤖 Agent Execution: ReAct-based Matching
For each formula, a dedicated FormulaAgent performs ReAct-style reasoning to map every variable in the formula to either:  

- a matching column name via a fuzzy semantic match, or
- an expression derived from existing columns (e.g., via ratio or arithmetic).

The agent internally leverages two subchains:  

- fuzzy_match — semantic alignment
- can_be_derived — compositional logic

In [31]:
from agents.formula_matcher_v0 import run_async, batch_match

results = run_async(batch_match(matcher_variables, columns))
import json; print(json.dumps(results, indent=2))


/Users/zhangran/Desktop/BP@UnitedStates/Code/D2D_Data2Dashboard/agents/formula_matcher_v0.py:116: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  raw = await fuzzy_chain.arun(var=var, columns=self.columns)


[
  {
    "formula": "(Gain_from_Investment - Cost_of_Investment) / Cost_of_Investment",
    "mapping": {
      "Gain_from_Investment": "Profit"
    },
    "status": "fail",
    "failure_reason": "derive_fail",
    "log": [
      {
        "step": "thought",
        "text": "Match variable 'Gain_from_Investment'"
      },
      {
        "step": "action",
        "tool": "fuzzy_match",
        "input": "Gain_from_Investment",
        "output": {
          "match": "Profit",
          "score": 0.8
        }
      },
      {
        "step": "thought",
        "text": "Match variable 'Cost_of_Investment'"
      },
      {
        "step": "action",
        "tool": "fuzzy_match",
        "input": "Cost_of_Investment",
        "output": {
          "match": "Profit",
          "score": 0.7
        }
      },
      {
        "step": "thought",
        "text": "Try derive 'Cost_of_Investment'"
      },
      {
        "step": "action",
        "tool": "can_be_derived",
        "input": "Cost_o